# Run this below cell only once 
as data will be deleted for each run

In [5]:
import os

def remove_lines(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    with open(file_path, 'w') as file:
        file.write(''.join(lines[10:]))

def remove_lines_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):  # Modify this condition based on the file extension
                file_path = os.path.join(root, file)
                remove_lines(file_path)

# Replace 'parent_folder_path' with the path to your parent folder
parent_folder_path = '3_Poolled_HAR'
remove_lines_in_folder(parent_folder_path)


In [6]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import random

# Function to convert a single text file to CSV with headers and set the "Activity" value
def txt_to_csv(input_file, output_file):
    activity_mapping = {
        "mar": 1,
        "pie": 2, 
        "sen": 3,
    }

    headers = ["RF", "BF", "VM", "ST", "Activity"]

    file_name = os.path.splitext(os.path.basename(input_file))[0]
    activity = activity_mapping.get(file_name[-3:])

    with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(headers)
        for line in txt_file:
            row = line.strip().split('\t')
            row.append(activity)
             # Check if the row has 7 columns
            if len(row) == 7:
                del row[5]  # Delete the 6th column
                row.insert(5, row.pop())  # Shift the 7th column to the 6th place
            if len(row) == 6:
                del row[4]
                row.insert(4,row.pop())
            csv_writer.writerow(row)

# Function to convert .txt files to .csv with headers and set the "Activity" value
def convert_txt_to_csv_in_subfolders(root_directory, output_stft_directory, sample_and_create_stft_func):
    for subfolder in os.listdir(root_directory):
        subfolder_path = os.path.join(root_directory, subfolder)
        if os.path.isdir(subfolder_path):
            # Check if the subfolder is a subject folder
                for filename in os.listdir(subfolder_path):
                    if filename.endswith('.txt'):
                        txt_file_path = os.path.join(subfolder_path, filename)
                        csv_file_path = os.path.splitext(txt_file_path)[0] + '.csv'
                        txt_to_csv(txt_file_path, csv_file_path)
                        print(f"Converted: {txt_file_path} to {csv_file_path}")
                        
                        # Sampling and STFT creation
                        sample_and_create_stft_func(csv_file_path, output_stft_directory)

# Function to sample CSV data randomly and create Short-Time Fourier Transform (STFT)
def sample_and_create_stft_random(csv_file_path, output_directory):
    df = pd.read_csv(csv_file_path)
    activity = df['Activity'].iloc[0]
    sensor_columns = ["RF", "BF", "VM", "ST"]

    sample_size = 1000
    num_samples = df.shape[0] // sample_size
    num_segments_per_channel = 30
    file_name = os.path.splitext(os.path.basename(csv_file_path))[0]

    for sensor in sensor_columns:
        for i in range(num_segments_per_channel):
            # Randomly select a start index within the valid range
            start_index = random.randint(0, num_samples - 1) * sample_size
            end_index = start_index + sample_size
            segment = df.loc[start_index:end_index - 1, sensor].values

            f, t, Zxx = signal.stft(segment, fs=1.0)  # Adjust nperseg as needed
            
            # Plot STFT without labels
            plt.figure(figsize=(5,5))
            plt.pcolormesh(t, f, np.abs(Zxx))  # Magnitude of STFT
            plt.axis('off')  # Turn off axis labels and ticks
            plt.gca().set_aspect('auto')  # Adjust aspect ratio for better visualization
            
            # Save STFT images in time-frequency format in activity-wise subfolders
            activity_output_directory = os.path.join(output_directory, f"Activity_{activity}")
            os.makedirs(activity_output_directory, exist_ok=True)
            
            stft_image_path = os.path.join(activity_output_directory, f"STFT_{file_name}_sensor{sensor}_random{i + 1}.png")
            plt.savefig(stft_image_path, bbox_inches='tight', pad_inches=0, transparent=True)
            plt.close()
            print(f"Saved STFT image: {stft_image_path}")

# Specify the root directory containing the subfolders and the directory to save STFT images
root_directory = '3_Poolled_HAR'
output_stft_directory = 'stft_HAR_30'  # Change this path

# Call the function to convert .txt files to .csv with headers and set the "Activity" value
convert_txt_to_csv_in_subfolders(root_directory, output_stft_directory, sample_and_create_stft_random)


Converted: 3_Poolled_HAR\1_WALKING\10Amar.txt to 3_Poolled_HAR\1_WALKING\10Amar.csv
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random1.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random2.png


<ipython-input-6-4c0c09de4f73>:75: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(t, f, np.abs(Zxx))  # Magnitude of STFT


Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random3.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random4.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random5.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random6.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random7.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random8.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random9.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random10.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random11.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random12.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random13.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random14.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorRF_random15.png
Saved 

Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random21.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random22.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random23.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random24.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random25.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random26.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random27.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random28.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random29.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Amar_sensorST_random30.png
Converted: 3_Poolled_HAR\1_WALKING\10Nmar.txt to 3_Poolled_HAR\1_WALKING\10Nmar.csv
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorRF_random1.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorRF_random

Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random8.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random9.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random10.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random11.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random12.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random13.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random14.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random15.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random16.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random17.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random18.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random19.png
Saved STFT image: stft_HAR_30\Activity_1.0\STFT_10Nmar_sensorST_random20.png
S

...

Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random9.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random10.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random11.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random12.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random13.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random14.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random15.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random16.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random17.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random18.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random19.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random20.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorRF_random21.png
Saved STFT im

Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorST_random28.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorST_random29.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Asen_sensorST_random30.png
Converted: 3_Poolled_HAR\3_STANDING\8Nsen.txt to 3_Poolled_HAR\3_STANDING\8Nsen.csv
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random1.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random2.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random3.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random4.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random5.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random6.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random7.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random8.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorRF_random9.png
Saved STFT im

Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random19.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random20.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random21.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random22.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random23.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random24.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random25.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random26.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random27.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random28.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random29.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_8Nsen_sensorST_random30.png
Converted: 3_Poolled_HAR\3_STANDING\9Asen.txt to 3_Poolled_HAR\3_STANDING\9Asen.csv
Save

Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random9.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random10.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random11.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random12.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random13.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random14.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random15.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random16.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random17.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random18.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random19.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random20.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Asen_sensorST_random21.png
Saved STFT im

Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorVM_random30.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random1.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random2.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random3.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random4.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random5.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random6.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random7.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random8.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random9.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random10.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random11.png
Saved STFT image: stft_HAR_30\Activity_3.0\STFT_9Nsen_sensorST_random12.png
Saved STFT image: stf